In [ ]:
from google.colab import files

files.upload()

In [4]:
import pandas as pd

Making a map from race ID to circuit ID

In [5]:
races_df= pd.read_csv('races.csv')
race_circuit_map= races_df[['raceId', 'circuitId']]

results_df: 'resultId','raceId','driverId','constructorId','number', 'grid','position', 'positionText', 'positionOrder, 'points', 'laps', 'time','milliseconds', 'fastestLap', 'rank', 'fastestLapTime','fastestLapSpeed', 'statusId'.

races_df: 'raceId', 'year', 'round', 'circuitId', 'name', 'date', 'time', 'url','fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time','quali_date', 'quali_time', 'sprint_date', 'sprint_time'

In [6]:
#1
results_df= pd.read_csv('results.csv')

merged_result_df = results_df.merge(race_circuit_map, on='raceId', how='left')

feature1 = merged_result_df.groupby(['driverId', 'circuitId'], as_index=False)['grid'].mean().rename(columns={'grid': 'avgGridPosition'})

feature1['avgGridPosition'] = feature1['avgGridPosition'].round(2)
feature1

,driverId,circuitId,avgGridPosition
0,1,1,4.29
1,1,2,5.09
2,1,3,4.22
3,1,4,4.37
4,1,5,6.29
...,...,...,...
8266,865,39,4.00
8267,865,70,13.00
8268,865,73,8.00
8269,865,77,14.00


In [7]:
#11
driver_races = results_df.merge(races_df[['raceId', 'circuitId', 'year', 'round']], on='raceId', how='left')
driver_races = driver_races.sort_values(['driverId', 'year', 'round'])
driver_races["previous_race"]= (driver_races.groupby(['driverId', 'circuitId']).cumcount())

feature11= driver_races[['driverId', 'circuitId', 'previous_race']]

feature11[feature11['driverId'] == 1]

,driverId,circuitId,previous_race
370,1,1,0
391,1,2,0
413,1,3,0
435,1,4,0
457,1,6,0
...,...,...,...
27004,1,13,18
27029,1,11,18
27057,1,39,4
27063,1,14,18


Momentum_df: 'raceId', 'driverId', 'lap', 'position', 'time', 'milliseconds'

In [8]:
#8
status_df= pd.read_csv('status.csv')
feature8= results_df.merge(status_df, on='statusId', how='left')
feature8['DNF']= feature8['status'].str.contains('Finished') == False
total_races = feature8.groupby('driverId')['raceId'].nunique().reset_index(name='total_races')
dnf_races = feature8[feature8['DNF']].groupby('driverId')['raceId'].nunique().reset_index(name='dnf_races')
driver_dnf_fraction = total_races.merge(dnf_races, on='driverId', how='left')
driver_dnf_fraction['dnf_races'] = driver_dnf_fraction['dnf_races'].fillna(0)
driver_dnf_fraction['dnf_fraction'] = driver_dnf_fraction['dnf_races'] / driver_dnf_fraction['total_races']

driver_dnf_fraction = driver_dnf_fraction.drop(["total_races",	"dnf_races"], axis=1)

driver_dnf_fraction

,driverId,dnf_fraction
0,1,0.126005
1,2,0.641304
2,3,0.296117
3,4,0.353919
4,5,0.651786
...,...,...
859,861,0.600000
860,862,0.571429
861,863,0.352941
862,864,0.470588


In [28]:
#12
total_laps = results_df.groupby("raceId")["laps"].max().reset_index()
total_laps.rename(columns={"laps": "total_laps"}, inplace=True)
results = results_df.merge(total_laps, on="raceId", how="left")
results["reliability_ratio"] = results["laps"] / results["total_laps"]

out= results[["driverId", "raceId", "reliability_ratio"]]
out

,driverId,raceId,reliability_ratio
0,1,18,1.000000
1,2,18,1.000000
2,3,18,1.000000
3,4,18,1.000000
4,5,18,1.000000
...,...,...,...
27093,807,1161,1.000000
27094,840,1161,1.000000
27095,842,1161,0.980392
27096,861,1161,0.980392


In [11]:
#Feature 13

pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")

pitStops= pit_stops.merge(race_circuit_map, on='raceId', how='left')
driver_race_pit_counts = pitStops.groupby(['driverId', 'raceId', 'circuitId']).size().reset_index(name='pitstop_count')

avg_pitstops_per_circuit = (driver_race_pit_counts.groupby(['driverId', 'circuitId'])['pitstop_count'].mean().reset_index().rename(columns={'pitstop_count': 'avg_pitstops_per_circuit'}))

avg_pitstops_per_circuit

,driverId,circuitId,avg_pitstops_per_circuit
0,1,1,1.916667
1,1,2,2.714286
2,1,3,2.285714
3,1,4,2.428571
4,1,5,2.000000
...,...,...,...
1737,865,39,2.000000
1738,865,70,2.000000
1739,865,73,1.000000
1740,865,77,1.000000


In [14]:
#15
def feature_avg_pos_diff_last_n(results_csv="results.csv", races_csv="races.csv", n=10, min_prev=1):
    # Load
    results = pd.read_csv(results_csv, usecols=["raceId","driverId","grid","positionOrder"])
    races   = pd.read_csv(races_csv,   usecols=["raceId","year","round"])

    # Order races chronologically per driver
    df = results.merge(races, on="raceId", how="left") \
                .sort_values(["driverId","year","round","raceId"]) \
                .reset_index(drop=True)

    # Per-race position change
    df["pos_diff"] = df["grid"] - df["positionOrder"]
    df.loc[df["grid"] == 0, "pos_diff"] = pd.NA  # ignore pit-lane/unknown starts

    # Rolling mean of the *previous* n races (no leakage via shift)
    roll_mean = (
        df.groupby("driverId", group_keys=False)["pos_diff"]
          .apply(lambda s: s.shift().rolling(window=n, min_periods=min_prev).mean())
    )

    df["avg_pos_diff_prev_lastN"] = roll_mean
    return df[["driverId","raceId","avg_pos_diff_prev_lastN"]]

feature_avg_pos_diff_last_n()

,driverId,raceId,avg_pos_diff_prev_lastN
0,1,36,NaN
1,1,37,1.00
2,1,38,1.50
3,1,39,1.00
4,1,40,1.25
...,...,...,...
27093,865,1157,-0.50
27094,865,1158,-1.60
27095,865,1159,-1.60
27096,865,1160,-1.90


In [17]:
#16
results = pd.read_csv("results.csv", usecols=["raceId","driverId","constructorId","points"])
races   = pd.read_csv("races.csv",   usecols=["raceId","year","round","circuitId"])

df = results.merge(races, on="raceId", how="left")

team_pts_race = (
    df.groupby(["year","round","raceId","constructorId"], as_index=False)["points"]
      .sum()
      .rename(columns={"points":"team_points_race"})
    .sort_values(["year","round","raceId"])
)

team_pts_race["cum_team_points_prev"] = (
    team_pts_race
    .groupby(["year","constructorId"], group_keys=False)["team_points_race"]
    .apply(lambda s: s.shift().cumsum())
)

team_pts_race["constructor_rank_prev"] = (
    team_pts_race
    .groupby(["year","round"])["cum_team_points_prev"]
    .rank(method="min", ascending=False)
)

team_pts_race["constructor_rank_prev"] = team_pts_race["constructor_rank_prev"].fillna(0).astype(int)

out = (
    df.merge(
        team_pts_race[["raceId","constructorId","constructor_rank_prev"]],
        on=["raceId","constructorId"], how="left"
    )[["driverId","raceId","circuitId","constructor_rank_prev"]]
)

out

,driverId,raceId,circuitId,constructor_rank_prev
0,1,18,1,0
1,2,18,1,0
2,3,18,1,0
3,4,18,1,0
4,5,18,1,0
...,...,...,...,...
27093,807,1161,73,8
27094,840,1161,73,6
27095,842,1161,73,10
27096,861,1161,73,10


In [19]:
#17
results = pd.read_csv("results.csv", usecols=["raceId","driverId","points"])
races   = pd.read_csv("races.csv",   usecols=["raceId","year","round","circuitId"])

df = results.merge(races, on="raceId", how="left")

driver_pts_race = (
    df.groupby(["year","round","raceId","driverId"], as_index=False)["points"]
      .sum()
      .rename(columns={"points":"driver_points_race"})
      .sort_values(["year","round","raceId"])
)

driver_pts_race["cum_driver_points_prev"] = (
    driver_pts_race
    .groupby(["year","driverId"], group_keys=False)["driver_points_race"]
    .apply(lambda s: s.shift().cumsum())
)

driver_pts_race["driver_rank_prev"] = (
    driver_pts_race
    .groupby(["year","round"])["cum_driver_points_prev"]
    .rank(method="min", ascending=False)
)

driver_pts_race["driver_rank_prev"] = driver_pts_race["driver_rank_prev"].fillna(0).astype(int)

out = (
    df.merge(
        driver_pts_race[["raceId","driverId","driver_rank_prev"]],
        on=["raceId","driverId"], how="left"
    )[["driverId","raceId","circuitId","driver_rank_prev"]]
)

out

,driverId,raceId,circuitId,driver_rank_prev
0,1,18,1,0
1,2,18,1,0
2,3,18,1,0
3,4,18,1,0
4,5,18,1,0
...,...,...,...,...
27093,807,1161,73,9
27094,840,1161,73,12
27095,842,1161,73,15
27096,861,1161,73,20


In [24]:
#23

def feature_points_last_n_seasons_on_circuit(results_csv="results.csv",
                                             races_csv="races.csv",
                                             n=3,
                                             min_seasons=1,
                                             fillna_with=0):
    results = pd.read_csv(results_csv, usecols=["raceId","driverId","points"])
    races   = pd.read_csv(races_csv,   usecols=["raceId","year","circuitId"])

    df = results.merge(races, on="raceId", how="left")

    dr_race_pts = (
        df.groupby(["raceId","driverId","year","circuitId"], as_index=False)["points"]
          .sum()
          .rename(columns={"points":"race_points"})
    )

    season_pts = (
        dr_race_pts.groupby(["driverId","circuitId","year"], as_index=False)["race_points"]
                   .sum()
                   .rename(columns={"race_points":"season_points"})
                   .sort_values(["driverId","circuitId","year"])
    )

    season_pts["points_last_n_seasons_on_circuit"] = (
        season_pts.groupby(["driverId","circuitId"], group_keys=False)["season_points"]
                  .apply(lambda s: s.shift().rolling(window=n, min_periods=min_seasons).sum())
    )

    out = (
        dr_race_pts.merge(
            season_pts[["driverId","circuitId","year","points_last_n_seasons_on_circuit"]],
            on=["driverId","circuitId","year"], how="left"
        )[["driverId","raceId","circuitId","points_last_n_seasons_on_circuit"]]
    )

    if fillna_with is not None:
        out["points_last_n_seasons_on_circuit"] = out["points_last_n_seasons_on_circuit"].fillna(fillna_with)

    return out

feature_points_last_n_seasons_on_circuit(n=10)

,driverId,raceId,circuitId,points_last_n_seasons_on_circuit
0,1,1,1,16.0
1,2,1,1,21.0
2,3,1,1,8.0
3,4,1,1,37.0
4,5,1,1,4.0
...,...,...,...,...
27002,860,1161,73,1.0
27003,861,1161,73,4.0
27004,863,1161,73,0.0
27005,864,1161,73,0.0


In [31]:

def feature_constructor_wins_last_n_seasons_on_circuit(results_csv="results.csv",
                                                       races_csv="races.csv",
                                                       n=3,
                                                       min_seasons=1,
                                                       fillna_with=0):

    results = pd.read_csv(results_csv, usecols=["raceId","driverId","constructorId","positionOrder"])
    races   = pd.read_csv(races_csv,   usecols=["raceId","year","circuitId"])

    df = results.merge(races, on="raceId", how="left")

    winners = (
        df[df["positionOrder"] == 1]
        .groupby(["raceId","constructorId"], as_index=False)
        .size()
        .rename(columns={"size":"is_win"})
    )
    winners["is_win"] = 1

    winners = winners.merge(races, on="raceId", how="left")
    cons_circuit_year_wins = (
        winners.groupby(["constructorId","circuitId","year"], as_index=False)["is_win"]
               .sum()
               .rename(columns={"is_win":"wins_in_season_on_circuit"})
               .sort_values(["constructorId","circuitId","year"])
    )

    cons_circuit_year_wins["constructor_wins_last_n_seasons_on_circuit"] = (
        cons_circuit_year_wins
        .groupby(["constructorId","circuitId"], group_keys=False)["wins_in_season_on_circuit"]
        .apply(lambda s: s.shift().rolling(window=n, min_periods=min_seasons).sum())
    )

    dr_race = df[["driverId","raceId","constructorId","year","circuitId"]].drop_duplicates()

    out = (
        dr_race.merge(
            cons_circuit_year_wins[["constructorId","circuitId","year","constructor_wins_last_n_seasons_on_circuit"]],
            on=["constructorId","circuitId","year"], how="left"
        )[["driverId","raceId","circuitId","constructor_wins_last_n_seasons_on_circuit"]]
    )

    if fillna_with is not None:
        out["constructor_wins_last_n_seasons_on_circuit"] = out["constructor_wins_last_n_seasons_on_circuit"].fillna(fillna_with)

    return out
feature_constructor_wins_last_n_seasons_on_circuit()

,driverId,raceId,circuitId,constructor_wins_last_n_seasons_on_circuit
0,1,18,1,3.0
1,2,18,1,0.0
2,3,18,1,0.0
3,4,18,1,0.0
4,5,18,1,3.0
...,...,...,...,...
27019,807,1161,73,0.0
27020,840,1161,73,0.0
27021,842,1161,73,0.0
27022,861,1161,73,0.0


In [38]:
#24, 25, 26
weather_df= pd.read_csv("weather_features_v4.csv")
weather_df

,temperature,precipitation,windspeed,round,name,datetime
0,15.8,0.0,18.8,1,British Grand Prix,1950-05-13 12:00:00
1,23.1,0.0,17.3,2,Monaco Grand Prix,1950-05-21 12:00:00
2,19.5,0.2,12.5,3,Indianapolis 500,1950-05-30 14:00:00
3,22.8,0.0,6.1,4,Swiss Grand Prix,1950-06-04 14:00:00
4,17.5,0.0,15.9,5,Belgian Grand Prix,1950-06-18 12:00:00
...,...,...,...,...,...,...
1120,20.1,0.1,12.6,20,Mexico City Grand Prix,2024-10-27 20:00:00
1121,20.6,2.5,15.2,21,São Paulo Grand Prix,2024-11-03 17:00:00
1122,10.7,0.0,1.1,22,Las Vegas Grand Prix,2024-11-23 06:00:00
1123,19.0,0.0,19.5,23,Qatar Grand Prix,2024-12-01 17:00:00


In [42]:
#2
def feature_laps_led_last_n(lap_times_csv="lap_times.csv",
                            results_csv="results.csv",
                            races_csv="races.csv",
                            n=5,
                            min_prev=1,
                            fillna_with=0):
    # Load
    lap_times = pd.read_csv(lap_times_csv, usecols=["raceId","driverId","lap","position"])
    results   = pd.read_csv(results_csv,   usecols=["raceId","driverId"])
    races     = pd.read_csv(races_csv,     usecols=["raceId","year","round","circuitId"])

    # Laps led per driver per race
    laps_led_race = (
        lap_times[lap_times["position"] == 1]
        .groupby(["raceId","driverId"], as_index=False)
        .size()
        .rename(columns={"size":"laps_led_in_race"})
    )

    # Include zeros for drivers who led none
    dr_race = (
        results.drop_duplicates()
               .merge(laps_led_race, on=["raceId","driverId"], how="left")
               .fillna({"laps_led_in_race": 0})
    )

    # Chronology + circuit
    df = (
        dr_race.merge(races, on="raceId", how="left")
               .sort_values(["driverId","year","round","raceId"])
               .reset_index(drop=True)
    )

    # Rolling sum over previous n races (no leakage)
    df["laps_led_lastN"] = (
        df.groupby("driverId", group_keys=False)["laps_led_in_race"]
          .apply(lambda s: s.shift().rolling(window=n, min_periods=min_prev).sum())
    )

    # Select output and avoid chained-assignment warnings
    out = df.loc[:, ["driverId","raceId","circuitId","laps_led_lastN"]].copy()
    if fillna_with is not None:
        out.loc[:, "laps_led_lastN"] = out["laps_led_lastN"].fillna(fillna_with)

    return out
feature_laps_led_last_n()

,driverId,raceId,circuitId,laps_led_lastN
0,1,36,1,0.0
1,1,37,2,4.0
2,1,38,3,6.0
3,1,39,4,10.0
4,1,40,6,18.0
...,...,...,...,...
27002,865,1157,13,0.0
27003,865,1158,11,0.0
27004,865,1159,39,0.0
27005,865,1160,14,0.0


In [43]:
#3
def feature_points_last_n(results_csv="results.csv",
                          races_csv="races.csv",
                          n=5,
                          min_prev=1,
                          fillna_with=0
                         ):

    results = pd.read_csv(results_csv, usecols=["raceId","driverId","points"])
    races   = pd.read_csv(races_csv,   usecols=["raceId","year","round","circuitId"])

    dr_race_pts = (
        results.groupby(["raceId","driverId"], as_index=False)["points"]
               .sum()
               .rename(columns={"points":"race_points"})
    )

    df = (
        dr_race_pts.merge(races, on="raceId", how="left")
                   .sort_values(["driverId","year","round","raceId"])
                   .reset_index(drop=True)
    )

    df["points_lastN"] = (
        df.groupby("driverId", group_keys=False)["race_points"]
          .apply(lambda s: s.shift().rolling(window=n, min_periods=min_prev).sum())
    )

    out = df.loc[:, ["driverId","raceId","circuitId","points_lastN"]].copy()
    if fillna_with is not None:
        out.loc[:, "points_lastN"] = out["points_lastN"].fillna(fillna_with)

    return out

feature_points_last_n()

,driverId,raceId,circuitId,points_lastN
0,1,36,1,0.0
1,1,37,2,6.0
2,1,38,3,14.0
3,1,39,4,22.0
4,1,40,6,30.0
...,...,...,...,...
27002,865,1157,13,14.0
27003,865,1158,11,6.0
27004,865,1159,39,0.0
27005,865,1160,14,15.0


In [44]:
#4
def feature_best_lap_prev_on_circuit(lap_times_csv="lap_times.csv",
                                     races_csv="races.csv",
                                     fillna_with=None):
    laps  = pd.read_csv(lap_times_csv, usecols=["raceId","driverId","lap","milliseconds"])
    races = pd.read_csv(races_csv,     usecols=["raceId","year","round","circuitId"])

    best_lap_race = (
        laps.dropna(subset=["milliseconds"])
            .groupby(["raceId","driverId"], as_index=False)["milliseconds"]
            .min()
            .rename(columns={"milliseconds": "best_lap_ms_in_race"})
    )

    df = (best_lap_race.merge(races, on="raceId", how="left")
                      .sort_values(["driverId","circuitId","year","round","raceId"])
                      .reset_index(drop=True))

    df["best_lap_ms_prev_on_circuit"] = (
        df.groupby(["driverId","circuitId"], group_keys=False)["best_lap_ms_in_race"]
          .apply(lambda s: s.shift().cummin())
    )

    out = df.loc[:, ["driverId","raceId","circuitId","best_lap_ms_prev_on_circuit"]].copy()
    if fillna_with is not None:
        out.loc[:, "best_lap_ms_prev_on_circuit"] = out["best_lap_ms_prev_on_circuit"].fillna(fillna_with)

    return out

feature_best_lap_prev_on_circuit()

,driverId,raceId,circuitId,best_lap_ms_prev_on_circuit
0,1,36,1,NaN
1,1,18,1,86351.0
2,1,1,1,86351.0
3,1,338,1,86351.0
4,1,841,1,86351.0
...,...,...,...,...
11363,865,1159,39,NaN
11364,865,1155,70,NaN
11365,865,1161,73,NaN
11366,865,1149,77,NaN


In [45]:
#5
def feature_driver_wins_prev_on_circuit(results_csv="results.csv",
                                        races_csv="races.csv",
                                        fillna_with=0):

    results = pd.read_csv(results_csv, usecols=["raceId","driverId","positionOrder"])
    races   = pd.read_csv(races_csv,   usecols=["raceId","year","round","circuitId"])

    dr_race = (
        results.groupby(["raceId","driverId"], as_index=False)["positionOrder"]
               .min()
               .rename(columns={"positionOrder":"best_finish"})
    )
    dr_race["is_win"] = (dr_race["best_finish"] == 1).astype(int)

    df = (dr_race.merge(races, on="raceId", how="left")
                .sort_values(["driverId","circuitId","year","round","raceId"])
                .reset_index(drop=True))

    df["driver_wins_prev_on_circuit"] = (
        df.groupby(["driverId","circuitId"], group_keys=False)["is_win"]
          .apply(lambda s: s.shift().cumsum())
    )

    out = df.loc[:, ["driverId","raceId","circuitId","driver_wins_prev_on_circuit"]].copy()
    if fillna_with is not None:
        out.loc[:, "driver_wins_prev_on_circuit"] = out["driver_wins_prev_on_circuit"].fillna(fillna_with)

    return out

feature_driver_wins_prev_on_circuit()

,driverId,raceId,circuitId,driver_wins_prev_on_circuit
0,1,36,1,0.0
1,1,18,1,0.0
2,1,1,1,1.0
3,1,338,1,1.0
4,1,841,1,1.0
...,...,...,...,...
27002,865,1159,39,0.0
27003,865,1155,70,0.0
27004,865,1161,73,0.0
27005,865,1149,77,0.0


In [46]:
#6
def feature_constructor_wins_prev_on_circuit(results_csv="results.csv",
                                             races_csv="races.csv",
                                             fillna_with=0):
    # Load
    results = pd.read_csv(results_csv, usecols=["raceId","driverId","constructorId","positionOrder"])
    races   = pd.read_csv(races_csv,   usecols=["raceId","year","round","circuitId"])

    # Winner constructor per race (any driver P1 → that constructor wins)
    winners = (
        results.loc[results["positionOrder"] == 1, ["raceId","constructorId"]]
               .drop_duplicates()
               .assign(is_win=1)
    )

    # Driver–race–constructor table + circuit + chronology
    dr_race = (
        results[["raceId","driverId","constructorId"]].drop_duplicates()
               .merge(races, on="raceId", how="left")
               .sort_values(["constructorId","circuitId","year","round","raceId"])
               .reset_index(drop=True)
    )

    # Attach win flag per race/constructor
    dr_race = dr_race.merge(winners, on=["raceId","constructorId"], how="left")
    dr_race["is_win"] = dr_race["is_win"].fillna(0).astype(int)

    # Cumulative wins on this circuit *before* this race (no leakage)
    dr_race["constructor_wins_prev_on_circuit"] = (
        dr_race.groupby(["constructorId","circuitId"], group_keys=False)["is_win"]
               .apply(lambda s: s.shift().cumsum())
    )

    # Final output
    out = dr_race.loc[:, ["driverId","raceId","circuitId","constructor_wins_prev_on_circuit"]].copy()
    if fillna_with is not None:
        out.loc[:, "constructor_wins_prev_on_circuit"] = out["constructor_wins_prev_on_circuit"].fillna(fillna_with)
    return out

feature_constructor_wins_prev_on_circuit()

,driverId,raceId,circuitId,constructor_wins_prev_on_circuit
0,57,224,1,0.0
1,14,224,1,0.0
2,14,207,1,0.0
3,57,207,1,1.0
4,57,191,1,2.0
...,...,...,...,...
27019,817,1126,79,0.0
27020,865,1150,79,0.0
27021,859,1150,79,0.0
27022,852,1142,80,0.0


In [47]:
#7
def feature_avg_qual_pos_last_n(qual_csv="qualifying.csv",
                                races_csv="races.csv",
                                n=5,            # lookback window (races)
                                min_prev=1,     # min prior races with a qual position
                                fillna_with=None):  # e.g., set to a large number if you prefer
    # Load
    qual  = pd.read_csv(qual_csv,  usecols=["raceId","driverId","position"])
    races = pd.read_csv(races_csv, usecols=["raceId","year","round","circuitId"])

    # One row per driver-race (defensive: if dup rows, take best/lowest position)
    dr_race_qual = (
        qual.groupby(["raceId","driverId"], as_index=False)["position"]
            .min()
            .rename(columns={"position":"qual_pos"})
    )

    # Attach chronology & circuit
    df = (
        dr_race_qual.merge(races, on="raceId", how="left")
                    .sort_values(["driverId","year","round","raceId"])
                    .reset_index(drop=True)
    )

    # Rolling mean of *previous* n qualifying positions (no leakage via shift)
    df["avg_qual_pos_lastN"] = (
        df.groupby("driverId", group_keys=False)["qual_pos"]
          .apply(lambda s: s.shift().rolling(window=n, min_periods=min_prev).mean())
    )

    # Output schema
    out = df.loc[:, ["driverId","raceId","circuitId","avg_qual_pos_lastN"]].copy()
    if fillna_with is not None:
        out.loc[:, "avg_qual_pos_lastN"] = out["avg_qual_pos_lastN"].fillna(fillna_with)

    return out

feature_avg_qual_pos_last_n()

,driverId,raceId,circuitId,avg_qual_pos_lastN
0,1,36,1,NaN
1,1,37,2,4.000000
2,1,38,3,4.000000
3,1,39,4,3.333333
4,1,40,6,3.500000
...,...,...,...,...
10829,865,1157,13,10.000000
10830,865,1158,11,10.400000
10831,865,1159,39,10.600000
10832,865,1160,14,9.600000
